### Lesson 1: Multi-Agent Conversation and Stand-up Comedy

#### Setup

In [11]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-4o-mini";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.17

Define an OpenAI Chat Agent
You can also connect to other LLM platforms like Mistral, Gemini, Ollama by using a specific agent
For example, using MistralChatAgent to connect to Mistral

In [12]:
var agent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "chatbot",
    modelName: openAIModel)
    .RegisterMessageConnector() // convert OpenAI Message to AutoGen Message
    .RegisterPrintMessage(); // print the message to the console

In [13]:
var _ = await agent.SendAsync("Tell me a joke");

from: chatbot
Why did the scarecrow win an award?

Because he was outstanding in his field!



#### Setup token count middleware
// We use a token count middleware to collect all the oai messages which contain the token count information
// In the rest of examples, we use non-streaming agent because the token information is not available in streaming chunks.

In [14]:
class CountTokenMiddleware : IMiddleware
{
    private readonly List<ChatCompletions> messages = new();
    public string? Name => nameof(CountTokenMiddleware);

    public int GetTokenCount()
    {
        return messages.Sum(m => m.Usage.CompletionTokens);
    }

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent agent, CancellationToken cancellationToken = default)
    {
        var reply = await agent.GenerateReplyAsync(context.Messages, context.Options, cancellationToken: cancellationToken);

        if (reply is IMessage<ChatCompletions> message)
        {
            messages.Add(message.Content);
        }

        return reply;
    }
}

var tokenCountMiddleware = new CountTokenMiddleware();

#### Conversation
Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [15]:
var openaiMessageConnector = new OpenAIChatRequestMessageConnector();
var cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: "Your name is Cathy and you are a stand-up comedian.")
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

var joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    Start the next joke from the punchline of the previous joke.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();


In [16]:
var chatResult = await joe.SendAsync(
    receiver: cathy,
    message: "I'm Joe. Let's keep the jokes rolling.",
    maxRound: 4);

TextMessage from cathy
--------------------
Hey Joe! I’m all about keeping the laughter alive! So, what did one ocean say to the other ocean? 

Nothing, they just waved! 

Got any favorite jokes of your own?
--------------------

TextMessage from joe
--------------------
Absolutely, Cathy! Speaking of waves, I once tried surfing... but all I caught was a cold! 

They say the ocean is good for your health, but I think it just gives you a new way to drown your sorrows!
--------------------

TextMessage from cathy
--------------------
Haha, Joe, that’s a good one! Surfing is a lot like dating – you paddle out, try to catch the perfect wave, and most of the time, you just wipe out! 

And you’re right about the ocean; it’s like a therapist. You go there to relax, but you always end up feeling a little seasick about your life choices! Got any more ocean-themed jokes up your sleeve?
--------------------

TextMessage from joe
--------------------
Oh, for sure, Cathy! You know, I once asked a f

Print token consumption

In [17]:
Console.WriteLine($"Total Token count: {tokenCountMiddleware.GetTokenCount()}");

Total Token count: 246


Get a better summary of conversation from a summary agent.

In [18]:
var summaryAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "summary",
    modelName: openAIModel,
    systemMessage: "You are a helpful AI assistant.")
    .RegisterMessageConnector()
    .RegisterPrintMessage();

await summaryAgent.SendAsync("summarize the converation", chatResult);

from: summary
Joe and Cathy are exchanging ocean-themed jokes and witty banter. Cathy starts with a joke about the ocean, and Joe responds with a surfing-related punchline. They continue sharing humorous observations about surfing and the ocean, with Cathy likening surfing to dating and discussing the ocean's therapeutic qualities. Joe adds another fish joke, emphasizing his preference for floating on a pool noodle rather than swimming. The conversation is light-hearted and focuses on humor related to the ocean and water activities.



#### Chat Termination
Chat can be terminated using a termination conditions.
Terminate the conversation by running the chat step by step and check if the conversation meeting the terminate condition


In [19]:
cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: """
    Your name is Cathy and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    """)
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    End the conversation when you see two jokes.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

In [20]:
var chatHistory = new List<IMessage>
{
    new TextMessage(Role.User, "I'm Joe. Let's keep the jokes rolling.", from: joe.Name)
};
var roundLeft = 10;
while(roundLeft > 0)
{
    var replies = await joe.SendAsync(
        receiver: cathy,
        chatHistory,
        maxRound: 1);

    var reply = replies.Last();
    if (reply.GetContent()?.ToLower().Contains("i gotta go") is true)
    {
        break;
    }
    chatHistory.Add(reply);
    roundLeft--;
}

TextMessage from cathy
--------------------
Hey Joe! Alright, let’s roll with some jokes! Why did the scarecrow win an award? Because he was outstanding in his field! What do you think? Got any favorites of your own?
--------------------

TextMessage from joe
--------------------
Hey Cathy! That's a classic! I love a good pun. Here’s one for you: Why don’t skeletons fight each other? They don’t have the guts! Got any more up your sleeve?
--------------------

TextMessage from cathy
--------------------
Oh, that's a good one, Joe! I love a pun that tickles the funny bone! Here’s another: Why did the bicycle fall over? Because it was two-tired! Got any more up your sleeve, or should I keep the jokes coming?
--------------------

TextMessage from joe
--------------------
That's a great one, Cathy! I love a good play on words. Alright, here’s another: Why did the math book look sad? Because it had too many problems! Keep ‘em coming or should I take a turn?
--------------------

TextMessage